### Multiscale Spatiotemporal Vision Transformer (MST-ViT | PixFormer)

### Misael M. Morales, 2024
***

In [ ]:
from heterogeneity import *

h = H2ViT()
h.__dict__

In [ ]:
h.load_data()

In [ ]:
h.train_model()

In [ ]:
h.plot_losses()

In [41]:
import numpy as np
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt
import time

In [16]:
data = np.load('train_dataF/realization_{}.npz'.format(100))
x, y = data['X_data'], data['y_data']
poro, perm, tstep = x[:,0], x[:,1], x[:,2]
pres, satu        = y[:,0], y[:,1]

print(x.shape, y.shape)

(60, 3, 256, 256) (60, 2, 256, 256)


In [ ]:
for i in range(1000):
    data = np.load('train_dataF/realization_{}.npz'.format(i))
    x, y = data['X_data'], data['y_data']
    poro, perm, tstep = x[:,0], x[:,1], x[:,2]
    pres, satu        = y[:,0], y[:,1]
    
    poro_n = np.expand_dims(cv2.resize(poro.T, (64,64)).T, 1)
    perm_n = np.expand_dims(cv2.resize(perm.T, (64,64)).T, 1)
    time_n = np.expand_dims(tstep[:,:64,:64], 1)

    pres_n = np.expand_dims(cv2.resize(pres.T, (64,64)).T, 1)
    satu_n = np.expand_dims(cv2.resize(satu.T, (64,64)).T, 1)

    X_data = np.concatenate([poro_n, perm_n, time_n], axis=1)
    y_data = np.concatenate([pres_n, satu_n], axis=1)

    np.savez('train_dataF_64x64/realization_{}'.format(i), X_data=X_data, y_data=y_data)

In [61]:
import numpy as np
import cv2
from concurrent.futures import ThreadPoolExecutor

In [62]:
def process_realization(i):
    data = np.load('train_dataF/realization_{}.npz'.format(i))
    x, y = data['X_data'], data['y_data']
    poro, perm, tstep = x[:,0], x[:,1], x[:,2]
    pres, satu = y[:,0], y[:,1]
    poro_n = np.expand_dims(cv2.resize(poro.T, (64,64)).T, 1)
    perm_n = np.expand_dims(cv2.resize(perm.T, (64,64)).T, 1)
    time_n = np.expand_dims(tstep[:,:64,:64], 1)
    pres_n = np.expand_dims(cv2.resize(pres.T, (64,64)).T, 1)
    satu_n = np.expand_dims(cv2.resize(satu.T, (64,64)).T, 1)
    X_data = np.concatenate([poro_n, perm_n, time_n], axis=1)
    y_data = np.concatenate([pres_n, satu_n], axis=1)
    np.savez('train_dataF_64x64/realization_{}'.format(i), X_data=X_data, y_data=y_data)

# Use ThreadPoolExecutor for parallel execution
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(process_realization, range(1000))

KeyboardInterrupt: 

In [60]:
np.load('temp.npz')['X_data'].shape

(60, 3, 64, 64)

***
# END